In [1]:
import scanpy as sc 
import numpy as np
import pandas as pd

In [2]:
adata = sc.read_h5ad('./data/Tcell_PDL1.h5ad')

In [3]:
cluster = np.unique(adata.obs['Sub_Cluster'])
cluster

array(['t_CD4-CXCL13', 't_CD4_Tcm-LMNA', 't_CD4_Tn-LEF1',
       't_CD4_Treg-FOXP3', 't_CD8-CXCL13', 't_CD8_MAIT-KLRB1',
       't_CD8_Teff-GNLY', 't_CD8_Tem-GZMK', 't_CD8_Trm-ZNF683',
       't_Tact-IFI6', 't_Tact-XIST', 't_Tprf-MKI67'], dtype=object)

In [4]:
for i in cluster:
    # select sub cluster
    data = adata[adata.obs['Sub_Cluster']==i]
    # read toomanycells cluster result
    tmc = pd.read_csv('./result/cluster/'+i+'_cluster.csv',usecols=['cell','cluster'],index_col=0)
    # add the toomanycells cluster information to 'adata.obs'
    data.obs = data.obs.join(tmc)
    # construct a DataFrame to store cluster count
    df = pd.DataFrame(data=np.unique(data.obs['cluster']),columns=['Cluster'])
    # calculate 'Count','PR_Count' and 'SD_Count' for each cluster
    for j in df.index:
        df_data = data.obs[data.obs['cluster']==df.loc[j,'Cluster']]
        df.loc[j,'Count'] = len(df_data)
        df.loc[j,'PR_Count'] = len(df_data[df_data['Response']=='PR'])
        df.loc[j,'SD_Count'] = len(df_data[df_data['Response']=='SD'])
    
    # add root information to 'df' and name it as cluster 0 
    df.loc[len(df)] = [0,len(data),len(data[data.obs['Response']=='PR']),len(data[data.obs['Response']=='SD'])]
    df = df.sort_values(by='Cluster')
    
    # caculate the Propotion
    df['PR_Prop'] = df['PR_Count'] / df['Count']
    df['SD_Prop'] = df['SD_Count'] / df['Count']
    # output the count 'df'
    df.to_csv('./data/cluster_count/'+i+'_cluster_count.csv',index=False)